# Basic Impact Evaluation

This notebook demonstrates the basic workflow for evaluating causal impact using the **Impact Engine**.

## Workflow Overview

1. Generate product characteristics using the catalog simulator
2. Configure the impact engine
3. Run impact evaluation
4. Review results

## Setup

Import the required packages.

In [ ]:
import json
from pathlib import Path

from impact_engine import evaluate_impact
from online_retail_simulator import simulate

## Step 1: Generate Product Characteristics

First, we generate a synthetic product catalog using the **Online Retail Simulator**.

In [ ]:
output_path = Path("output/demo_basic")
output_path.mkdir(parents=True, exist_ok=True)

job_info = simulate("configs/demo_basic_catalog.yaml", job_id="catalog")
products = job_info.load_df("products")

print(f"Generated {len(products)} products")
print(f"Products catalog: {job_info.get_store().full_path('products.csv')}")
products.head()

## Step 2: Configure Impact Engine

Create a configuration file that specifies:
- **DATA**: Where to get products and how to simulate metrics
- **MEASUREMENT**: Which statistical model to use

In [ ]:
config_path = "configs/demo_basic.yaml"

## Step 3: Run Impact Evaluation

Call `evaluate_impact()` with the configuration file. The engine handles:
- Loading products
- Simulating daily metrics
- Aggregating data
- Running the interrupted time series model

In [ ]:
results_path = evaluate_impact(config_path, str(output_path), job_id="results")
print(f"Results saved to: {results_path}")

## Step 4: Review Results

Load and display the impact evaluation results.

In [ ]:
with open(results_path, "r") as f:
    results = json.load(f)

data = results["data"]
model_params = data["model_params"]
estimates = data["impact_estimates"]
summary = data["model_summary"]

print("=" * 50)
print("IMPACT EVALUATION RESULTS")
print("=" * 50)
print(f"\nModel: {results['model_type']}")
print(f"Intervention Date: {model_params['intervention_date']}")
print(f"Dependent Variable: {model_params['dependent_variable']}")

print("\n--- Impact Estimates ---")
print(f"Pre-intervention mean:  ${estimates['pre_intervention_mean']:,.2f}")
print(f"Post-intervention mean: ${estimates['post_intervention_mean']:,.2f}")
print(f"Absolute change:        ${estimates['absolute_change']:,.2f}")
print(f"Percent change:         {estimates['percent_change']:.1f}%")

print("\n--- Model Summary ---")
print(f"Observations:   {summary['n_observations']}")
print(f"Pre-period:     {summary['pre_period_length']} days")
print(f"Post-period:    {summary['post_period_length']} days")